In [10]:
# breakhistf2v026-nsl.ipynb  v1.0.0 
# 
#  --------------------------------------------------
#  Hangzhou Domain Zones Technology Co., Ltd

#  Apache Licence 2.0       https://www.apache.org/licenses/LICENSE-2.0
#  --------------------------------------------------


import os
import tensorflow as tf
import numpy as np
import csv
import matplotlib.pyplot as plt
from tensorflow import keras
import neural_structured_learning as nsl
from tensorflow.python.keras.api._v2.keras import layers, optimizers, losses
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import TensorBoard
import random

tf.random.set_seed(22)
np.random.seed(22)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
assert tf.__version__.startswith('2.')

IMAGE_INPUT_NAME = 'images'
LABEL_INPUT_NAME = 'labels'
  
%matplotlib inline

def convert_to_dictionaries(image, label):
    return {IMAGE_INPUT_NAME:image, LABEL_INPUT_NAME:label}

def read_csv(csvnamepath, filename):
     

    # read from csv file
    images, labels = [], []
    with open(os.path.join(csvnamepath, filename)) as f:
        reader = csv.reader(f)
        for row in reader:
            img, label = row
            label = int(label)

            images.append(img)
            labels.append(label)

    assert len(images) == len(labels)

    return images, labels

 
def load_breakhis(dirc, mode='train'):
    classdir2label={}
    filename="tf2breakhisCSV"
    filedirs = os.listdir( dirc)
    for filedir in filedirs:
        if not os.path.isdir(os.path.join(dirc,filedir)):
            continue
        classdir2label[filedir]=len(classdir2label.keys())
    

     
    images, labels = read_csv(os.path.join(os.path.abspath('.'),'tf2breakhis'), filename )  
    if mode == 'train':  # 60%
        images = images[:int(0.6 * len(images))]
        labels = labels[:int(0.6 * len(labels))]
    elif mode == 'val':  # 20% = 60%->80%
        images = images[int(0.6 * len(images)):int(0.8 * len(images))]
        labels = labels[int(0.6 * len(labels)):int(0.8 * len(labels))]
    else:  # 20% = 80%->100%
        images = images[int(0.8 * len(images)):]
        labels = labels[int(0.8 * len(labels)):]
    return images, labels, classdir2label
 
img_mean = tf.constant([0.485, 0.456, 0.406])
img_std = tf.constant([0.229, 0.224, 0.225])

@tf.function
def normalize(x, mean=img_mean, std=img_std):
    # x shape: [224, 224, 3]
    # mean：shape为1；这里用到了广播机制。我们安装好右边对齐的原则，可以得到如下；
    # mean : [1, 1, 3], std: [3]        先插入1
    # mean : [224, 224, 3], std: [3]    再变为224
    x = (x - mean)/std
    return x

# 数据normalize之后，这里有一个反normalizaion操作。比如数据可视化的时候，需要反过来。
# @tf.function
def denormalize(x, mean=img_mean, std=img_std):
    x = x * std + mean
    return x

 


# 预处理的函数，复制过来。
# @tf.function
def preprocessnsl(images,labels):
    i =0;
    images1= []
    labels1= []
    imagelen = len(images)
    #imagelen = 20
    print (len(images))
    while (True):
        if i >= imagelen :
            break
        
        x= images[i]
        y =labels[i]
        x = tf.io.read_file(x)
     
        x = tf.image.decode_jpeg(x, channels=3) # RGBA
    
        x = tf.image.resize(x, [244, 244])
   
   
        #x = tf.image.random_flip_left_right(x)
    
        x = tf.image.random_crop(x, [224,224,3])
   
        x = tf.cast(x, dtype=tf.float32) / 255.
        x = normalize(x)
        y = tf.convert_to_tensor(y)
       # y = tf.one_hot(y, depth=8)
        
        images1.append(x)
        labels1.append(y)
        i = i+1 
      
    
    return images1, labels1

def convert_to_tensor(images,labels):
    timages1 = tf.convert_to_tensor(images)
    #timages1.reshape(imagelen,2224,224,3) 
    tlabels1 = tf.convert_to_tensor(labels) 

    return timages1, tlabels1

@tf.function
def preprocess(x,y):
    # x: 图片的路径，y：图片的数字编码
   
    x = tf.io.read_file(x)
     
    x = tf.image.decode_jpeg(x, channels=3) # RGBA
    
    x = tf.image.resize(x, [244, 244])
   
   
    x = tf.image.random_flip_left_right(x)
    #plt.imshow(x)
  
    # x = tf.image.random_flip_up_down(x)
    x = tf.image.random_crop(x, [224,224,3])
    #plt.imshow(x)
    
    # x: [0,255]=> -1~1
    x = tf.cast(x, dtype=tf.float32) / 255.
    x = normalize(x)
    y = tf.convert_to_tensor(y)
    y = tf.one_hot(y, depth=8)

    return x, y



# 预处理的函数，复制过来。
@tf.function
def preprocess1(x,y):
    # x: 图片的路径，y：图片的数字编码
   
    x = tf.io.read_file(x)
     
    x = tf.image.decode_jpeg(x, channels=3) # RGBA
    
    x = tf.image.resize(x, [244, 244])
   
    
    # x = tf.image.random_flip_up_down(x)
    x = tf.image.random_crop(x, [224,224,3])
      
    # x: [0,255]=> -1~1
    x = tf.cast(x, dtype=tf.float32) / 255.
    x = normalize(x)
    y = tf.convert_to_tensor(y)
    y = tf.one_hot(y, depth=8)

    return x, y


 

def shuffletwolists(list1,list2):
    temp = list(zip(list1, list2))
    random.shuffle(temp)
    res1 ,res2 =zip(*temp)
    return list(res1), list(res2)

 

batchsz = 32
epochnum =10
# creat train db   一般训练的时候需要shuffle。其它是不需要的。
images, labels, table = load_breakhis(os.path.join(os.path.abspath('.'),'tf2breakhis'), 'train')

 

(x_train , y_train)  =preprocessnsl(images ,labels ) 

 
(x_train , y_train)  =shuffletwolists( x_train ,y_train ) 

(x_train , y_train)  = convert_to_tensor(x_train , y_train)
# crate validation db
images2, labels2, table = load_breakhis(os.path.join(os.path.abspath('.'),'tf2breakhis'), 'val')

#db_val = tf.data.Dataset.from_tensor_slices((images2, labels2))
#db_val = db_val.shuffle(1000).repeat(2).map(preprocess).shuffle(1000).batch(batchsz)


(x_val , y_val)  = preprocessnsl(images2 ,labels2 ) 

(x_val , y_val)  =shuffletwolists( x_val ,y_val ) 
(x_val , y_val)  =convert_to_tensor( x_val ,y_val ) 

# create test db

images3, labels3, table = load_breakhis(os.path.join(os.path.abspath('.'),'tf2breakhis'), 'test')
#db_test = tf.data.Dataset.from_tensor_slices((images3, labels3))
#db_test = db_test.map(preprocess1).batch(batchsz)

(x_test , y_test)  = preprocessnsl(images3 ,labels3 )
(x_test , y_test)  = convert_to_tensor(x_test , y_test)
 
resnetdense = keras.Sequential([

    tf.keras.Input((224, 224,3), name='feature'),
    tf.keras.layers.Conv2D(16,5,3),
    tf.keras.layers.MaxPool2D(3,3),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Conv2D(64,5,3),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.ReLU(),
    
    tf.keras.layers.Flatten(),
   
    tf.keras.layers.Dense(128), 
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(64),  
    tf.keras.layers.ReLU(),
    #tf.keras.layers.Dense(8)
    tf.keras.layers.Dense(8, activation=tf.nn.softmax)
   ])


base_model = tf.keras.Sequential([
    tf.keras.Input((224, 224,3), name='feature'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
 




resnetdense.build(input_shape=(batchsz, 224, 224, 3))
resnetdense.summary()

 
resnetdense.compile(optimizer=optimizers.Adam(lr=1e-4),
               loss=losses.CategoricalCrossentropy(from_logits=True),
               metrics=['accuracy'])
 

LOGDIR='log/breakhistf2v026' 


#resnetdense.fit(db_train, validation_data=db_val, validation_freq=10, epochs=1, callbacks=[TensorBoard(log_dir=LOGDIR)])  
#resnetdense.fit({'feature': x_train, 'label': y_train}, validation_data={'feature': x_val, 'label': y_val}, validation_freq=10, epochs=1, callbacks=[TensorBoard(log_dir=LOGDIR)])  
 
#resnetdense.evaluate(db_test)
 
 

 
adv_grad_norm = 'infinity'

#adv_config = nsl.configs.make_adv_reg_config(multiplier=0.2, adv_step_size=0.05, adv_grad_norm = adv_grad_norm) 
#adv_model = nsl.keras.AdversarialRegularization(base_model, adv_config)# Compile, train, and evaluate. 


# Wrap the model with adversarial regularization.
adv_config = nsl.configs.make_adv_reg_config(multiplier=0.2, adv_step_size=0.05,adv_grad_norm = adv_grad_norm)
adv_model = nsl.keras.AdversarialRegularization(resnetdense, adv_config=adv_config)

# Compile, train, and evaluate.
adv_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
#print(db_train.shape)
#adv_model.fit(db_train ,   epochs=5)
#adv_model.evaluate(db_test)

adv_model.fit({'feature': x_train, 'label': y_train}, batch_size=32, epochs=500)
adv_model.evaluate({'feature': x_test, 'label': y_test})


#adv_model.compile(optimizer=optimizers.Adam(lr=1e-4), loss=losses.sparse_categorical_crossentropy(from_logits=True), metrics=[‘accuracy’]) 


#db_train_for_adv_model = db_train.map(convert_to_dictionaries)

#db_val_for_adv_model = db_val.map(convert_to_dictionaries)

#db_test_for_adv_model = db_test.map(convert_to_dictionaries)

  


#adv_model.compile(optimizer=optimizers.Adam(lr=1e-4),
      #         loss=losses.SparseCategoricalCrossentropy(from_logits=True),
      #         metrics=['accuracy'])

#adv_model.fit(db_train, epochs=5) 

#adv_model.fit(db_train_for_adv_model, validation_data=db_val_for_adv_model, use_multiprocessing=False,  validation_freq=10, epochs=3000, callbacks=[TensorBoard(log_dir=LOGDIR)])  

#adv_model.fit(db_train_for_adv_model,  epochs=30  )  

#adv_model.evaluate(db_test_for_adv_model)

  
 
 
    
    
 

4745
1582
1582
Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 74, 74, 16)        1216      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 24, 24, 16)        0         
_________________________________________________________________
re_lu_12 (ReLU)              (None, 24, 24, 16)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 7, 7, 64)          25664     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 3, 3, 64)          0         
_________________________________________________________________
re_lu_13 (ReLU)              (None, 3, 3, 64)          0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 576

4745/4745 [==============================] - 60s 13ms/sample - loss: 0.6080 - sparse_categorical_crossentropy: 0.2693 - sparse_categorical_accuracy: 0.9144 - adversarial_loss: 1.7000
Epoch 73/500
4745/4745 [==============================] - 57s 12ms/sample - loss: 0.6029 - sparse_categorical_crossentropy: 0.2659 - sparse_categorical_accuracy: 0.9157 - adversarial_loss: 1.6832
Epoch 74/500
4745/4745 [==============================] - 59s 12ms/sample - loss: 0.6124 - sparse_categorical_crossentropy: 0.2770 - sparse_categorical_accuracy: 0.9128 - adversarial_loss: 1.6918
Epoch 75/500
4745/4745 [==============================] - 52s 11ms/sample - loss: 0.5977 - sparse_categorical_crossentropy: 0.2648 - sparse_categorical_accuracy: 0.9134 - adversarial_loss: 1.6717
Epoch 76/500
4745/4745 [==============================] - 58s 12ms/sample - loss: 0.6013 - sparse_categorical_crossentropy: 0.2654 - sparse_categorical_accuracy: 0.9168 - adversarial_loss: 1.6715
Epoch 77/500
4745/4745 [=========

4745/4745 [==============================] - 74s 16ms/sample - loss: 0.4092 - sparse_categorical_crossentropy: 0.1259 - sparse_categorical_accuracy: 0.9699 - adversarial_loss: 1.4145
Epoch 155/500
4745/4745 [==============================] - 92s 19ms/sample - loss: 0.3969 - sparse_categorical_crossentropy: 0.1195 - sparse_categorical_accuracy: 0.9715 - adversarial_loss: 1.3857
Epoch 156/500
4745/4745 [==============================] - 77s 16ms/sample - loss: 0.4326 - sparse_categorical_crossentropy: 0.1459 - sparse_categorical_accuracy: 0.9572 - adversarial_loss: 1.4392
Epoch 157/500
4745/4745 [==============================] - 75s 16ms/sample - loss: 0.4279 - sparse_categorical_crossentropy: 0.1408 - sparse_categorical_accuracy: 0.9633 - adversarial_loss: 1.4514
Epoch 158/500
4745/4745 [==============================] - 70s 15ms/sample - loss: 0.4477 - sparse_categorical_crossentropy: 0.1535 - sparse_categorical_accuracy: 0.9585 - adversarial_loss: 1.4781
Epoch 159/500
4745/4745 [====

4745/4745 [==============================] - 57s 12ms/sample - loss: 0.3474 - sparse_categorical_crossentropy: 0.0975 - sparse_categorical_accuracy: 0.9772 - adversarial_loss: 1.2482
Epoch 237/500
4745/4745 [==============================] - 59s 12ms/sample - loss: 0.3405 - sparse_categorical_crossentropy: 0.0913 - sparse_categorical_accuracy: 0.9777 - adversarial_loss: 1.2495
Epoch 238/500
4745/4745 [==============================] - 58s 12ms/sample - loss: 0.3309 - sparse_categorical_crossentropy: 0.0881 - sparse_categorical_accuracy: 0.9802 - adversarial_loss: 1.2137
Epoch 239/500
4745/4745 [==============================] - 56s 12ms/sample - loss: 0.3419 - sparse_categorical_crossentropy: 0.0929 - sparse_categorical_accuracy: 0.9798 - adversarial_loss: 1.2465
Epoch 240/500
4745/4745 [==============================] - 60s 13ms/sample - loss: 0.3369 - sparse_categorical_crossentropy: 0.0877 - sparse_categorical_accuracy: 0.9800 - adversarial_loss: 1.2470
Epoch 241/500
4745/4745 [====

4745/4745 [==============================] - 60s 13ms/sample - loss: 0.3137 - sparse_categorical_crossentropy: 0.0835 - sparse_categorical_accuracy: 0.9793 - adversarial_loss: 1.1501
Epoch 319/500
4745/4745 [==============================] - 57s 12ms/sample - loss: 0.3157 - sparse_categorical_crossentropy: 0.0836 - sparse_categorical_accuracy: 0.9798 - adversarial_loss: 1.1591
Epoch 320/500
4745/4745 [==============================] - 59s 13ms/sample - loss: 0.3236 - sparse_categorical_crossentropy: 0.0861 - sparse_categorical_accuracy: 0.9798 - adversarial_loss: 1.1931
Epoch 321/500
4745/4745 [==============================] - 61s 13ms/sample - loss: 0.3113 - sparse_categorical_crossentropy: 0.0835 - sparse_categorical_accuracy: 0.9802 - adversarial_loss: 1.1354
Epoch 322/500
4745/4745 [==============================] - 56s 12ms/sample - loss: 0.2916 - sparse_categorical_crossentropy: 0.0672 - sparse_categorical_accuracy: 0.9863 - adversarial_loss: 1.1206
Epoch 323/500
4745/4745 [====

4745/4745 [==============================] - 59s 12ms/sample - loss: 0.2918 - sparse_categorical_crossentropy: 0.0696 - sparse_categorical_accuracy: 0.9838 - adversarial_loss: 1.1106
Epoch 401/500
4745/4745 [==============================] - 55s 12ms/sample - loss: 0.2837 - sparse_categorical_crossentropy: 0.0693 - sparse_categorical_accuracy: 0.9852 - adversarial_loss: 1.0710
Epoch 402/500
4745/4745 [==============================] - 58s 12ms/sample - loss: 0.2925 - sparse_categorical_crossentropy: 0.0742 - sparse_categorical_accuracy: 0.9821 - adversarial_loss: 1.0883
Epoch 403/500
4745/4745 [==============================] - 58s 12ms/sample - loss: 0.2733 - sparse_categorical_crossentropy: 0.0620 - sparse_categorical_accuracy: 0.9852 - adversarial_loss: 1.0559
Epoch 404/500
4745/4745 [==============================] - 62s 13ms/sample - loss: 0.2742 - sparse_categorical_crossentropy: 0.0644 - sparse_categorical_accuracy: 0.9857 - adversarial_loss: 1.0464
Epoch 405/500
4745/4745 [====

4745/4745 [==============================] - 69s 14ms/sample - loss: 0.2770 - sparse_categorical_crossentropy: 0.0655 - sparse_categorical_accuracy: 0.9850 - adversarial_loss: 1.0554
Epoch 483/500
4745/4745 [==============================] - 76s 16ms/sample - loss: 0.2699 - sparse_categorical_crossentropy: 0.0588 - sparse_categorical_accuracy: 0.9857 - adversarial_loss: 1.0522
Epoch 484/500
4745/4745 [==============================] - 104s 22ms/sample - loss: 0.2588 - sparse_categorical_crossentropy: 0.0575 - sparse_categorical_accuracy: 0.9874 - adversarial_loss: 1.0121
Epoch 485/500
4745/4745 [==============================] - 67s 14ms/sample - loss: 0.2664 - sparse_categorical_crossentropy: 0.0619 - sparse_categorical_accuracy: 0.9848 - adversarial_loss: 1.0227
Epoch 486/500
4745/4745 [==============================] - 64s 13ms/sample - loss: 0.2597 - sparse_categorical_crossentropy: 0.0612 - sparse_categorical_accuracy: 0.9846 - adversarial_loss: 0.9910
Epoch 487/500
4745/4745 [===

[2.993924109583106, 1.5289712, 0.66055626, 7.287493]